## SMAVA

Test exercise - Question 2 - Data investigation

Reduce the features used and try other models on whole of the dataset or sub-sets for each bank.

### Overview

#### Acceptance

Firstly, we analyze and decide that an acceptance tester for each bank can be 64% accurate using a radial SVM. 
A logistic model is only 60%. (Cross-validated within the train set.)

We note that this is with a reduced feature set: which is invariant across all banks - ie. the model enables each bank to decide with the same data on the customer. No conditioning on x6 or x7.

#### Interest Rate

There are two category fields, x6 and x7 which may help decide the interest rate offered by bank.
x6 is two-valued, x7 is 2000!

We should also revisit the correlations. x1 x10 and x4 are good for acceptance, but the discarded x values could correlate for interest rate.


In [2]:
## weaves
## smava

getwd()

## load in packages
library(tidyverse)
library(rpart.plot)

library(e1071)
library(MASS)
library(mlbench)
library(gbm)
library(kernlab)
library(RSNNS)

## last loaded, first found methods
library(caret)

## implementation
library(doMC)

registerDoMC(cores = detectCores(all.tests = FALSE, logical = TRUE))

options(useFancyQuotes = TRUE)

[1] "/misc/build/0/pyeg0/caret/classr1"

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: rpart


Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select


Loaded gbm 2.1.5


Attaching package: 'kernlab'


The following object is masked from 'package:purrr':

    cross


The following object is masked from 'package:ggplot2':

    alpha


Loading required package: Rcpp

Loading required package: lattice


Attaching package: 'caret'


The following objects are masked from 'package:RSNNS':

    confusionMatrix, train


The following object is masked from 'package:purrr':

    lift


Loading required package: foreach


Attaching package:

In [3]:
load("bak/in/train.rdata")
load("bak/in/test.rdata")

train0 <- data.frame(train) # a local copy.

In [ ]:
## Logistic regression for acceptance using Narrow Feature set with conditioning on bank over all banks 
method = 'polr' # fails in jupyter-lab, 59% standalone
method = 'plr' # fails in jupyter-lab
method = 'multinom' # fails in jupyter-lab

method = 'glm' # 0.59

fit1 <- train(
  form = accepted ~ x1 + x10 + x4 + bank,
  data = train,
  trControl = trainControl(method = "cv", number = 5),
  method = method,
  family = "binomial"
)
fit1

In [4]:
## Not very good. Let's try a predictor specific to a bank
## List the banks.
smava1 <- list()
smava1$banks <- unique(train[["bank"]])
length(smava1$banks)
smava1$banks

[1] 30

[1] B14 B4  B25 B3  B1  B8  B18 B21 B6  B15 B16 B2  B12 B27 B7  B30 B9  B22 B28
[20] B17 B26 B23 B11 B20 B19 B13 B29 B24 B5  B10
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

In [5]:
## Random choose a bank to look at
fidx <- function(N0, n=3) sample(1:N0, n, replace = FALSE)

t <- smava1$banks
tag <- t[fidx(length(t))]
tag <- tag[1:1]
tag
colnames(train)

[1] B3
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

[1] "customerNumber" "bank"           "x1"             "x2"            
 [5] "x3"             "x4"             "x5"             "x6"            
 [9] "x7"             "x8"             "x9"             "x10"           
[13] "accepted"       "interestRate"

In [3]:
load("bak/in/train.rdata")
load("bak/in/test.rdata")

train0 <- data.frame(train) # a local copy.

In [6]:
cols <- colnames(train)
cols <- cols[grep('^x', cols)]
cols <- setdiff(cols, "x3")
xcols <- c(cols, "accepted")
xcols

[1] "x1"       "x2"       "x4"       "x5"       "x6"       "x7"      
 [7] "x8"       "x9"       "x10"      "accepted"

In [7]:
## Specific to a bank
## And impute x2
tag
outcomes <- train[ train$bank %in% tag, "accepted"]

train1 <- train[ train$bank %in% tag, xcols]

train1$x2na <- 0
train1[is.na(train1$x2), "x2na"] <- 1

v0 <- train1[train1$x2 & train1$x2na == 0, "x2"]
v0 <- mean(v0[["x2"]])

train1[is.na(train1$x2), "x2"] <- v0

dim(train1)
head(train1)

# test1[ is.na(test1$x2), "x2" ] <- smava0$x2impute

[1] B3
30 Levels: B1 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B2 B20 B21 B22 ... B9

[1] 4480   11

x1,x2,x4,x5,x6,x7,x8,x9,x10,accepted,x2na
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1.7044089,5.4123865,-1.209511,1.8524490,FBW,PBSEO,0.7316942,0.2678961,0.3731034,NO,0
6.2399036,3.9923303,3.601843,-0.6324472,FBW,O07XX,3.2772413,0.1890368,0.3876183,YES,0
0.6404203,0.4344275,-1.211129,-1.3633386,ACS,QTDO0,-2.5410318,0.2141143,0.3487595,NO,0
2.8531209,7.9625419,4.341369,4.7253100,FBW,QUTOJ,9.0522801,0.2389058,0.4001331,YES,1
0.7511392,7.9625419,5.903780,7.7164806,ACS,49CT6,13.8247432,0.3056381,0.3548651,YES,1
0.3687868,7.9625419,-3.454459,5.5783330,FBW,IEWGU,1.9566097,0.8271761,0.2675379,NO,1


In [ ]:
## Logistic fit for one bank is barely any better.
## Don't add to the formula x7 - no-converge

fit1x <- train(
  form = accepted ~ x1 + x10 + x4,
  data = train1,
  trControl = trainControl(method = "cv", number = 5),
  method = "glm",
  family = "binomial"
)
fit1x

### Note

Tried a few formulae, no improvement with x2. 
No real improvement even if limited to just one bank.

Try some SVM classifiers - these only converge on small data-sets. There are a number of 

A pair plot of features how a couple of X-Y central scattering, suggest that radials can be good method

In [ ]:
## Formally constrain to the Narrow Feature set
cols <- c("x1", "x10", "x4", "accepted")
df0 <- train1[,cols]

In [ ]:
plot(df0)

In [ ]:
## Heuristically try other models
## 0.65 accurate by bank for acceptance with just these three x.

method <- 'svmLinear2'
method <- 'svmPoly'

## all these give 64%
method <- 'svmRadial'
method <- 'svmRadialSigma'
method <- 'svmRadialCost'

In [ ]:
## By bank - acceptance using Narrow Feature set

fit1x <- train(
  form = accepted ~ .,
  data = df0,
  trControl = trainControl(method = "cv", number = 5),
  method = method    
)
fit1x

In [ ]:
trainPred <- predict(fit1x, df0)
# postResample(testPred, testClass)

conf0 <- confusionMatrix(trainPred, df0[["accepted"]], positive = "YES")
conf0

### Recap and Plan: customer to acceptances at each bank

Model with predictor trained by bank using 'svmRadialCost' gets 64% on the training set - cross-validated.
using just the fields: "x1", "x10", "x4" - the Narrow Feature Set

So for each customer, we want to know what banks might accept them. 

A customer gets a different set of values from each bank, but not on the features of the Narrow Feature set.
So the customer record can be duplicated for each bank. A prediction made with that bank's predictor for all customers.

That would produce the banks.rdata results without any methodological issues, but that is unordered.

### Plan: bank ordering by interest rate

We then have to order the bank choice by interest rate given. We should train a second regression predictor, for all accepted customers at a bank and the interest they receive. And we have to consider if x6 and x7 are correlated to interest rate.

In [ ]:
## Visual - check the feature relationships - customer is constant across bank. Interest rate varies.
cols <- unique(c(cols, "x1", "x4", "x10"))
df2 <- train[order(train$customerNumber, train$bank), c("customerNumber", "accepted", "bank", c(cols, "x6", "x7"), "x2", "interestRate")]
head(df2, n=50)

In [ ]:
## Logistic regression using Narrow Feature set with conditioning on bank over all banks.

fit1 <- train(
  form = accepted ~ x1 + x10 + x4,
  data = train,
  trControl = trainControl(method = "cv", number = 5),
  method = "glm",
  family = "binomial"
)
fit1

In [ ]:
## An interest rate predictor would be like this. 
df3 <- df2[df2$accepted == "YES", c("bank", "interestRate", "customerNumber", "x7", "x6", cols)]
df3[order(-df3$interestRate, df3$bank),]

## Built for each bank it would take the x values, returning interestRate for all 

The customer numbers at all banks, so a weighted regression on x values to interestRate 
with a two-ply tree on x6 and x7.
Banks are 30 in number so same number of predictors. Try a global regression predictor using x6, x7 and xvalues

In [8]:
# Look at interest rates for one bank.
df4 <- train[train$bank == tag & train$accepted == "YES", setdiff(colnames(train), c("customerNumber", "bank", "accepted", "x3", "x6", "x7"))]

x2bar <- mean(df4$x2, trim=0, na.rm=TRUE)
x2bar
df4[is.na(df4$x2), "x2"] <- x2bar

[1] 7.655432

In [ ]:
plot(df4)

In [ ]:
## All over looks of business
summary(df4)

In [ ]:
## Nothing new on corrrelations

c0 <- cor(df4)
corrplot::corrplot(c0, method="number", order="hclust")

In [ ]:
## Look for a regression and check Rsquared

regressControl  <- trainControl(method="repeatedcv",
                    number = 4,
                    repeats = 5
                    ) 

## R^2 of only 0.3 is not good enough, with x6 is not better.
regress <- train(interestRate ~ .,
           data = df4,
           method  = "lm",
           trControl = regressControl)
regress

In [ ]:
## Let's overfit with a multi-layer perceptron and see if I can use x6 and x7.
## Regression model two inaccurate. A simple multi-layer perceptron - a look up table almost.

fControl  <- trainControl(method="repeatedcv",
                    number = 4,
                    repeats = 5
                    )
method = 'mlp'
method = 'mlpWeightDecay'

mi <- getModelInfo(model = method, regex = FALSE)[[1]]
p0 <- data.frame(mi$parameters)
p0

In [ ]:
data.frame(interaction.depth = c(1, 5, 9))

In [10]:
## "x6", "x7"
df4 <- train[train$bank == tag & train$accepted == "YES", setdiff(colnames(train), c("customerNumber", "bank", "accepted", "x3"))]

x2bar <- mean(df4$x2, trim=0, na.rm=TRUE)
x2bar
df4[is.na(df4$x2), "x2"] <- x2bar
head(df4)

[1] 7.655432

x1,x2,x4,x5,x6,x7,x8,x9,x10,interestRate
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
6.2399036,3.992330,3.601843,-0.6324472,FBW,O07XX,3.277241,0.1890368,0.3876183,12.215344
2.8531209,7.655432,4.341369,4.7253100,FBW,QUTOJ,9.052280,0.2389058,0.4001331,3.348640
0.7511392,7.655432,5.903780,7.7164806,ACS,49CT6,13.824743,0.3056381,0.3548651,2.666739
2.7489449,4.250198,3.325267,1.9186531,FBW,GPX9G,5.926024,0.8271125,0.5347677,3.336138
6.9003717,4.110479,-5.173110,0.8183836,FBW,WXATN,-4.403885,0.3888524,0.4672193,14.509473
0.4255057,14.477147,1.607090,-3.3508678,ACS,4AME2,-1.415343,0.5023046,0.3325895,2.348572


In [ ]:
## Try a simple neural network - with 11 hidden layers, you can get R^2 to 0.11 - awful.

fit0 <- train(interestRate ~ x1 + x10 + x7 + x6,
           data = df4,
           method  = method,
           trControl = fControl, tuneGrid = data.frame(size=c(7)))
fit0

In [14]:
head(df4)

x1,x2,x4,x5,x6,x7,x8,x9,x10,interestRate
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
6.2399036,3.992330,3.601843,-0.6324472,FBW,O07XX,3.277241,0.1890368,0.3876183,12.215344
2.8531209,7.655432,4.341369,4.7253100,FBW,QUTOJ,9.052280,0.2389058,0.4001331,3.348640
0.7511392,7.655432,5.903780,7.7164806,ACS,49CT6,13.824743,0.3056381,0.3548651,2.666739
2.7489449,4.250198,3.325267,1.9186531,FBW,GPX9G,5.926024,0.8271125,0.5347677,3.336138
6.9003717,4.110479,-5.173110,0.8183836,FBW,WXATN,-4.403885,0.3888524,0.4672193,14.509473
0.4255057,14.477147,1.607090,-3.3508678,ACS,4AME2,-1.415343,0.5023046,0.3325895,2.348572


## Try an SVM regression model

This is developed in smava03.ipynb.

The final configuration is 60% accurate, by bank using svmRadial

In [ ]:
# just use train1 no preProcess, just these features.

fit0 <- train(interestRate ~ x1 + x10 + x4,
           data = train1,
           method = 'svmRadial',
              preProcess = c("zv"),
#           tuneGrid = fGrid, 
           trControl = fControl)
fit0

## Note

There's a lot of variation at the bank for the interest rate. It may be that one of the factor fields is a product.
The codes in x6 and x7 might be multi-modal with respective to the x6 (or x7).

So I should check the pair-plot with interestRate.

In [ ]:
## check x6 and x7 - chi-sq by eye
## No clear classification on interestRate

## Note
## One day, I'll learn dylpr
tag <- "x6"
tag1 <- "ACS"

tag <- "x7"
tags <- unique(train[[tag]])
length(tags)
tag1<- tags[2]

cols <- c("x10", "bank", "x5", tag, "interestRate")
df3 <- train[!is.na(train$interestRate), cols]
df3 <- df3[ order(df3$bank, df3[[tag]]), cols]
# lapply(df3[ df3$bank == "B1",], summary)

lapply(df3[ df3[[tag]] == tag1 & df3$bank == "B1",], summary)
lapply(df3[ df3[[tag]] != tag1 & df3$bank == "B1",], summary)

## x7 product codes

Is the x7 a product field that is common to all banks?

It looks like bank B9 has never offered a particular x7

In [ ]:
x7s <- c("69TO8", "157H7")
x7s0 <- unique(train[["x7"]])
length(x7s0)
df0 <- data.frame(with(train, table(x7, x6)))
df0[order(-df0$Freq),]
df0 <- data.frame(with(train, table(x7, bank)))
df0[order(-df0$Freq),]

In [ ]:
summary(train$x6)
summary(train$x7)
with(train, table(x6, x7))

In [ ]:
## No obvious splits?
cp0 <- 0.005
method0 <- "class" # anova
method0 <- "anova" # anova
x7s
tag <- "WXATN"
tag <- x7s[1]
df0 <- train[train$x7 == tag, ]
tag
tree <- rpart(interestRate ~ bank + x7 + x6, data=df0, cp=cp0, method = method0)
                         # cp=.02 because want small tree for demo

rpart.plot(tree)

## x6 FBW and ACS

In [ ]:
with(warpbreaks, table(wool, tension))

In [ ]:
with(train, table(bank, x7))

In [ ]:
## Interesting products
## Trying to see if any relationship between x7 and interestRate. Very sparse data, two codes have more than 20 per bank.
## Possibly a standard product?
x7s <- c("69TO8", "157H7")

In [ ]:
df0 <- train[!is.na(train$interestRate) & train$x7 %in% x7s, c("bank", "interestRate", "x7")]

f <- function(x) list(summary(x))

by_cyl <- df0 %>% group_by(bank) %>%  summarise(
  disp = f(interestRate),
)
by_cyl

by_cyl <- df0 %>% group_by(x7) %>%  summarise(
  disp = f(interestRate),
)
by_cyl


In [ ]:
summary(1:10)